# LightGBMの推論用カスタムコンテナを構築し、SageMakerによる推論の仕組みを深く理解する

## アジェンダ
* パターン4のカスタムコンテナを作成
    * 学習編で実施済み
* パターン2のカスタムコンテナを作成：コンテナファイルを外部から指定する
    * Inference-Toolkitと、MMSをインストールしたコンテナを作成
* XGBoostビルトインコンテナで、LightBGMの推論を実施する
    * requirements.txtを用意する
    * inference.pyをお作用に従って作る

# Lab : LightGBMのカスタムコンテナを通して、SageMakerの動作を理解する

LightGBMがインストールされたカスタムコンテナを構築し、SageMaker Trainingジョブで学習を行います。
カスタムコンテナの挙動を観察し、SageMakerの動作について理解を深めます。

ノートブックは20分程度で実行できます。

# 0.実行環境確認
本ノートブックは、SageMakerノートブックインスタンス上で動作確認しています。
* インスタンスタイプ：ml.t3.medium
* カーネル：conda_python3

## 0-1.pythonバージョン確認

In [ ]:
#Pythonのバージョン情報
import sys
sys.version # 3.8.12

In [ ]:
# Pythonのバージョン確認 (システムコマンド使用）
!python -V # 3.8.12

## 0-2.SageMakerSDKバージョン確認

Amazon SageMaker Python SDKは、Amazon SageMaker上で機械学習されたモデルをトレーニングおよびデプロイするためのオープンソースライブラリです。

このSDKを使用すると、一般的な深層学習フレームワーク、Amazonが提供するアルゴリズム、またはSageMaker互換のDockerイメージに組み込まれた独自のアルゴリズムを使ってモデルをトレーニングおよびデプロイすることができます。

* ドキュメント : https://sagemaker.readthedocs.io/en/stable/
* GitHub : https://github.com/aws/sagemaker-python-sdk

SageMakerSDK をインポートすると、バケットが作成されます。  
sagemaker-＜region＞-＜account＞

In [ ]:
# SageMakerSDK のバージョン確認
import sagemaker
print('Current SageMaker Python SDK Version ={0}'.format(sagemaker.__version__)) # 2.110.0

# 1.データ準備

学習、推論で利用するデータを準備します。

scikit-learn付属の、ボストン住宅価格データセットを利用します。(注：バージョン1.2から除外されます）  
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html

以下のスクリプトを参考にしています。

https://github.com/aws-samples/amazon-sagemaker-local-mode/blob/main/lightgbm_bring_your_own_container_local_training_and_serving/lightgbm_bring_your_own_container_local_training_and_serving.py

In [ ]:
import sklearn
sklearn.__version__ # 1.0.1

In [ ]:
import pandas as pd
pd.__version__ # 1.3.4

## 1-1. データロード

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [ ]:
data = load_boston() # 1.2でデータセットがなくすという警告が出ますが動作に影響ありません

## 1-2. 特徴量生成（Feature Engineering）
本ノートブックでは実施しません。そのままデータを利用します。

## 1-3. データ分割
学習用（train）、評価用（validation）、テスト用（test）にデータを分割します。  
train:val:test = 3(60%):1(20%):1(20%)に分割します。  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=45)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=45)

trainX = pd.DataFrame(X_train, columns=data.feature_names)
trainX['target'] = y_train

valX = pd.DataFrame(X_val, columns=data.feature_names)
valX['target'] = y_val

testX = pd.DataFrame(X_test, columns=data.feature_names)

In [ ]:
# 確認
trainX.head()

In [ ]:
# 確認
valX.head()

In [ ]:
# 確認
testX.head()

In [ ]:
# 確認
y_test[0:5]

## 1-4.データ保存
ローカル、S3それぞれにデータを保存します。

### 1-4-1.ローカルへ保存

In [ ]:
# ディレクトリ作成
from pathlib import Path

Path('./data/train').mkdir(parents=True, exist_ok=True)
Path('./data/valid').mkdir(parents=True, exist_ok=True)
Path('./data/test').mkdir(parents=True, exist_ok=True)

In [ ]:
# ローカルへ保存
local_train = './data/train/boston_train.csv'
local_valid = './data/valid/boston_valid.csv'
local_test = './data/test/boston_test.csv'

trainX.to_csv(local_train, header=None, index=False)
valX.to_csv(local_valid, header=None, index=False)
testX.to_csv(local_test, header=None, index=False)

### 1-4-2.S3へ保存

一意のバケット作成のために、sgemaker.Session().default_bucket()を利用します。

https://sagemaker.readthedocs.io/en/stable/api/utility/session.html#sagemaker.session.Session

sagemaker-＜region＞-＜accoutid＞　を取得することができます。

In [ ]:
bucket_name = sagemaker.Session().default_bucket()
region_name = sagemaker.Session().boto_region_name
account_id =  sagemaker.Session().account_id()

In [ ]:
# 確認
print(bucket_name)
print(region_name)
print(account_id)

In [ ]:
# バケット作成(SageMakerSDKのインポート時作成されています。他のバケット作成時に利用ください)
#import boto3

#s3_resource = boto3.resource('s3')
#s3_resource.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region_name})

In [ ]:
# S3へ保存
train_s3 = sagemaker.s3.S3Uploader.upload('./data/train/boston_train.csv', f's3://{bucket_name}/demo_lightgbm/train')
valid_s3 = sagemaker.s3.S3Uploader.upload('./data/valid/boston_valid.csv', f's3://{bucket_name}/demo_lightgbm/valid')

# 2.LightGBMカスタムコンテナの構築


カスタムコンテナの作成には大きく分けて3つのパターンがあります。詳細は以下のブログを参考ください。

https://aws.amazon.com/jp/blogs/news/sagemaker-custom-containers-pattern-training/

SageMakerの動作を理解するためにパターン3のベースイメージ + カスタムレイヤー方式を採用します。

## 2-1. Dockerfileの確認

資材はこちらのノートブックを参考に準備しています。

https://github.com/aws-samples/amazon-sagemaker-local-mode/tree/main/lightgbm_bring_your_own_container_local_training_and_serving/container

In [ ]:
!pygmentize ./container/Dockerfile

SageMakerのモデルデプロイは、SageMaker SDKでは、deploy()で実行します。

その際に、SageMakerは

docker run < image > server

を実行します。

この場合、以下の serve スクリプトが実行されます。

In [ ]:
!pygmentize -l py ./container/lightgbm_regression/serve

mainからstart_server()を実行します。

start_server()では、
* nginxの起動（webサーバの役割）
* gunicornの起動（application サーバの役割）
    * gunicornの起動コマンド引数に'wsgi:app'とあるように、wsgiモジュールwsgi.pyの、appアプリケーションを読み込みます。

が行われます。
nginxの起動の際に、nginx.confが読み込まれます。

In [ ]:
!pygmentize ./container/lightgbm_regression/nginx.conf

gunicornへのアプリケーションのキック様に使われるファイル。

gunicornの起動コマンド引数に'wsgi:app'とあるように、wsgiモジュールwsgi.pyの、appアプリケーションを読み込みます。

In [ ]:
!pygmentize ./container/lightgbm_regression/wsgi.py

predictor.pyの、appを読み込みます。

In [ ]:
!pygmentize ./container/lightgbm_regression/predictor.py

Flaskアプリケーションを構成していることがわかりました。では、これらをビルドしていきましょう。

## 2-2. dockerイメージの build & push

ビルド&pushには7分ほどかかります。

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-lightgbm-regression

cd container

chmod +x lightgbm_regression/train
chmod +x lightgbm_regression/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to ap-northeast-1 if none defined)
region=$(aws configure get region)
region=${region:-ap-northeast-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

## 2-3. 学習前設定
ECRでpushしたコンテナのURIを確認

AWSコンソールでECRに移動し、作成したコンテナがあることを確認します。

image URIを取得し、以下にはりつけます。

In [ ]:
# 確認
print(bucket_name)
print(region_name)
print(account_id)

In [ ]:
image_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/sagemaker-lightgbm-regression'

In [ ]:
# 確認
image_uri

In [ ]:
hyperparameters={'boosting_type': 'gbdt',
            'objective': 'regression',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0}

## 2-4.ローカル学習の実行
ECRからビルドしたイメージを持ってきて、ローカルのdockerでビルドして、実行する

In [ ]:
# ローカルファイルのパスを設定（S3パス指定も可）
train_location = 'file://'+local_train
valid_location = 'file://'+local_valid

print(train_location)
print(valid_location)

In [ ]:
from sagemaker.estimator import Estimator

In [ ]:
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
# 確認
role

SageMakerのEstimatorを作成します。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html

In [ ]:
local_lightgbm = Estimator(
    image_uri,
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters
    )

fitメソッドで学習ジョブを発行します

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.fit

In [ ]:
local_lightgbm.fit({'train':train_location, 'validation': valid_location})

ローカルモードの学習結果は

Amazon S3
Buckets
sagemaker-us-west-2-805433377179
sagemaker-lightgbm-regression-2022-10-03-06-17-32-054/

に出力されます。


## 2-5.ローカルデプロイ

serializer : インプットデータの形式を指定します。
https://sagemaker.readthedocs.io/en/stable/v2.html

In [ ]:
local_predictor = local_lightgbm.deploy(1, 'local', serializer=sagemaker.serializers.CSVSerializer()) 

In [ ]:
!docker ps

In [ ]:
# 起動中のコンテナを停止する場合
#!docker stop XXXXXXXXXXX #XXXXXXXXXXXは CONtAINER ID
#!docker ps

## 2-6.ローカルエンドポイントで推論実施

In [ ]:
# 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = local_predictor.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

## 2-7.学習ジョブを発行
次は、ローカルモードではなく、
同じカスタムコンテナで、学習ジョブを実行します。

In [ ]:
# 確認
print(train_s3)
print(valid_s3)

In [ ]:
est_lightgbm = Estimator(
    image_uri,
    role,
    instance_count=1,
    instance_type="ml.m4.2xlarge", # インスタンスタイプを指定
    hyperparameters=hyperparameters)

In [ ]:
est_lightgbm.fit({'train':train_s3, 'validation': valid_s3})

学習には3分ほど時間がかかります。

課金されるのは75秒ほどです。

## 2-8.エンドポイントにデプロイ

デプロイすると、
SageMaker は docker run <image> serveを実行します。

    
デプロイには3分ほどかかります。

In [ ]:
from sagemaker.predictor import csv_serializer

deployメソッドで、推論エンドポイントをデプロイします。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.deploy

In [ ]:
predictor = est_lightgbm.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer, wait=True)

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor.predict(payload).decode('utf-8')
print(predicted)

# 3.Inference-toolkitの導入。実行ファイルを外部から指定する。Appサーバーまわりの制御はSageMakerの仕組みを用いる。

自身で nginx, gunicorn, Flask の仕組みを構築することは、メンテナンスコストが発生します。
Inference-toolkitを導入することで、すでに用意された環境を利用することができます。

* Inference-Toolkitを導入する
* ビルトインコンテナ + requirements.txt, inference.pyを利用する

本セクションでは、Inference-Toolkitを導入します。

https://github.com/aws/sagemaker-inference-toolkit

サンプルコード

Complete Example
Here is a complete example demonstrating usage of the SageMaker Inference Toolkit in your own container for deployment to a multi-model endpoint.

https://github.com/aws/amazon-sagemaker-examples/tree/main/advanced_functionality/multi_model_bring_your_own

In [ ]:
!pygmentize ./container_sminftoolkit/Dockerfile

これは、以下に該当する。

3.Implement a serving entrypoint, which starts the model server.


https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/model_server.py

start_model_server()は、引数指定しない場合、

DEFAULT_HANDLER_SERVICE = default_handler_service.__name__

を指定。これは、inference-toolkitのハンドラサービスである。

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/default_handler_service.py



ハンドラサービスが、Transformer()を作り、そのなかで、推論ハンドラが作られている。

DefaultHandlerService -> Transformer -> DefaultInferenceHandler

https://github.com/aws/sagemaker-inference-toolkit/blob/3774c1a0fb4408cfa95333b75d6e30a376bffa52/src/sagemaker_inference/transformer.py


In [ ]:
!pygmentize ./container_sminftoolkit/dockerd-entrypoint.py

start_model_server()は引数指定しない場合、
inference-toolkitのTransform()が作られる。

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/model_server.py

DEFAULT_HANDLER_SERVICE = default_handler_service.__name__

より、

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/default_handler_service.py

__init__にて、Trransformer()が実行

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/transformer.py

Transform()において、inference-toolkitのDefaultInferenceHandlerが利用される。

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/default_inference_handler.py

よって、このdockerd-entrypoint.pyが最小構成となる。

## 解説
ハンドラサービスと推論ハンドラがある。

ハンドラサービスは、以下に該当する。

2.Implement a handler service that is executed by the model server.

モデルの推論ハンドラは、以下に該当する。

1.Implement an inference handler, which is responsible for loading the model and providing input, predict, and output functions. 


2.のハンドラサービスから、1.の推論ハンドラがロードされる。推論ハンドラはinference-toolkitで用意したものを使ってもよい。

In [ ]:
#!pygmentize ./container_sminftoolkit/model_handler.py ### 最小構成には不要

In [ ]:
%%sh

# The name of our algorithm
#algorithm_name=demo-sagemaker-multimodel
algorithm_name=demo-sagemaker-inftoolkit

#cd container
cd container_sminftoolkit

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -q -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

* dockerd-entrypoint.py が実行され、サーバーの起動を試みる。
    * サーバー起動の際に必要はハンドラーは、odel-handler.pyに記載されている。
    


## ローカルにエンドポイントをデプロイ
モデルは前のセクションで作成したLGBMモデル

* ソースも指定する
* LGBMはrequirements.txtでインストールする

In [ ]:
container_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/demo-sagemaker-inftoolkit:latest'

In [ ]:
container_uri

In [ ]:
### 2.8の学習ジョブで構築したモデルを利用する
est_lightgbm.model_data

In [ ]:
!docker ps

In [ ]:
#全コンテナ停止
!docker stop $(docker ps -q)

In [ ]:
!docker ps

In [ ]:
from sagemaker.predictor import RealTimePredictor

lgb_model = sagemaker.model.Model(#est_xgb.image_uri, # XGBoostビルトインコンテナのURI
                                  container_uri,
                                  model_data=est_lightgbm.model_data, # ローカル学習で生成したモデルファイル
                                  role=role,
                                  predictor_cls=RealTimePredictor, # 推論するための識別子を指定
                                  source_dir='./src_builtin_container_serve', # requirements.txt必要な場合
                                  entry_point='inference.py' # source_dirを指定している場合、.pyファイルを指定する。
                                  #entry_point='./src_builtin_container_serve/inference.py'
                                 )

In [ ]:
predictor_lgb_model = lgb_model.deploy(initial_instance_count=1,
                                       instance_type='local', 
                                       serializer=csv_serializer, ### string形式でSageMakerに渡す（認識してもらう）
                                      )

In [ ]:
!docker ps

In [ ]:
#!docker stop f380dc891702

In [ ]:
#!docker ps

## 推論実施

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor_lgb_model.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

In [ ]:
print(predicted)

In [ ]:
type(predicted)

## 返り値をstr以外で受け取りには
Deserializerの説明

# (optional) XGBoostコンテナで、LGBMの推論を実施する

LGBMのカスタムコンテナも存在する
< URL >
    

In [ ]:
xgb_container_uri = sagemaker.image_uris.retrieve("xgboost", region_name, "1.5-1")

In [ ]:
xgb_container_uri

In [ ]:
from sagemaker.predictor import RealTimePredictor

lgb_model = sagemaker.model.Model(xgb_container_uri, # XGBoostビルトインコンテナのURI
                                  model_data=est_lightgbm.model_data, # ローカル学習で生成したモデルファイル
                                  role=role,
                                  predictor_cls=RealTimePredictor, # 推論するための識別子を指定
                                  source_dir='./src_builtin_container_serve', # requirements.txt必要な場合
                                  entry_point='inference.py' # source_dirを指定している場合、.pyファイルを指定する。
                                 )

In [ ]:
!docker ps

In [ ]:
!docker stop $(docker ps -q)

In [ ]:
!docker ps

In [ ]:
predictor_lgb_model = lgb_model.deploy(initial_instance_count=1,
                                       instance_type='local', 
                                       serializer=csv_serializer, ### string形式でSageMakerに渡す（認識してもらう）
                                       #deserializer=None, 
                                      )

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor_lgb_model.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

In [ ]:
print(predicted)

# END of Containts =======================

# 後片付け

# 参考

## （optional）4. カスタムコンテナを使わず、built-inコンテナのrequirement.txtにlightgbmを記載して実行する



過去バージョン（1.3-3, 1.2-2, 1.2-1, 1.0-1)はこちら

https://github.com/aws/sagemaker-xgboost-container/releases


## 4-2. 推論実施

### 4-2-1.デプロイ

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.deploy


デプロイの際に、ソースコードを指定するにはどうしたらいいのか？

https://www.youtube.com/watch?v=sngNd79GpmE&t=596s


ポイント：あらためて、Estimatorを定義する必要がある。

### serve用のファイルは、.py かつ、作法に従う必要がある。

MMSは.pyを扱うように設計されているため。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator

## エラー

RuntimeError: Model /opt/ml/model/model.tar.gz cannot be loaded:


6o0805unb3-algo-1-k8ugv | [2022-10-16 02:25:50 +0000] [19] [ERROR] Exception in worker process  
6o0805unb3-algo-1-k8ugv | Traceback (most recent call last):  
6o0805unb3-algo-1-k8ugv |   File "/miniconda3/lib/python3.8/site-packages/sagemaker_xgboost_container/algorithm_mode/serve_utils.py", line 175, in get_loaded_booster  
6o0805unb3-algo-1-k8ugv |     booster = pkl.load(open(full_model_path, "rb"))  
6o0805unb3-algo-1-k8ugv | _pickle.UnpicklingError: invalid load key, '\x1f'.  

## 原因
lightgbm-regression-model.txtなので、pklでは読み込めない。

モデルロードする関数を上書きするには？？（そもそもこれがやりたい）

https://github.com/aws/sagemaker-xgboost-container/blob/master/docker/1.5-1/final/Dockerfile.cpu

# Set SageMaker entrypoints
ENV SAGEMAKER_TRAINING_MODULE sagemaker_xgboost_container.training:main  
ENV SAGEMAKER_SERVING_MODULE sagemaker_xgboost_container.serving:main  


まず、serving.main()が実行される

https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/serving.py

L143

serving_env = env.ServingEnv()

で、環境変数にパラメータが読み込まれる


L147

user_module = modules.import_module(serving_env.module_dir, serving_env.module_name)

ここで、ユーザーのモジュールが読み込まれる。

L18をみると、sagemaker_containers.beta.framework.modulesがモジュールのようだ。

from sagemaker_containers.beta.framework import (
    encoders,
    env,
    modules,
    server,
    transformer,
    worker,
)

https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/beta/framework/__init__.py

sagemaker_containers.beta.frameworkはアーカイブされている。

現在はこちら。initをみると

https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_modules.py



L258で、imortしている。

module = importlib.import_module(name)

def import_module(uri, name=DEFAULT_MODULE_NAME, cache=None):  # type: (str, str, bool) -> module

とあるように、DEFAULT_MODULE_NAMEが読み込まれるようだ





https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/serving.py

L148,149: L147で読み込んだユーザーモジュールに上書きする

user_module_transformer = _user_module_transformer(user_module)  
user_module_transformer.initialize()  


L116にあるように、model_fnなどのユーザー関数に上書きされる。


def _user_module_transformer(user_module):  
    model_fn = getattr(user_module, "model_fn", default_model_fn)  
    input_fn = getattr(user_module, "input_fn", None)  
    predict_fn = getattr(user_module, "predict_fn", None)  
    output_fn = getattr(user_module, "output_fn", None)  
    transform_fn = getattr(user_module, "transform_fn", None)  

## model_fnを定義したファイルが、importされているか？

いま、そもそも環境変数に正しく情報渡せていない気がする。


https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_modules.py

L237より、

def import_module(uri, name=DEFAULT_MODULE_NAME, cache=None):  # type: (str, str, bool) -> module

第二引数に指定する必要がある。

これを呼ぶのは、


https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/serving.py

L147

user_module = modules.import_module(serving_env.module_dir, serving_env.module_name)

serving_env.module_name である。指定できているのか？


L143より

serving_env = env.ServingEnv()

これは、以下のファイル。

https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_env.py

L862

class ServingEnv(_Env):



https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_env.py

L329には、

class _Env(_mapping.MappingMixin):


module_name = os.environ.get(_params.USER_PROGRAM_ENV, None)

とある。


L595

TrainingEnvには、

        # override base class attributes  
        if self._module_name is None:  
            self._module_name = str(sagemaker_hyperparameters.get(_params.USER_PROGRAM_PARAM, None))  
        self._user_entry_point = self._user_entry_point or sagemaker_hyperparameters.get(  
            _params.USER_PROGRAM_PARAM  
        )  
        
        
        

## USER_PROGRAM_ENVに設定できればいい？


https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/parameters.py


L18

USER_PROGRAM_ENV = "SAGEMAKER_PROGRAM"  # type: str

SAGEMAKER_PROGRAMに設定できればいいようだ。

ビルトインコンテナにはどうすれば設定できるのだろうか？？

以下のYouTubeだと、boto3でEnvironment引数を使っている。

https://youtu.be/sngNd79GpmE?t=780

# デバッグのために、dockerイメージをプルして、中をみてみる。

ビルトインコンテナの中身をみるには、どうすればいいのか？

XGBoostの場合は、ローカルでbuildしていくようだ。

https://github.com/aws/sagemaker-xgboost-container

# コンテナの中に入って確認する方法
コンソールを立ち上げて、以下の流れで実行する


ディレクトリ移動
 $ cd sagemaker-xgboost-container/
 
baseコンテナをビルド
 $ docker build -t xgboost-container-base:1.5-1-cpu-py3 -f docker/1.5-1/base/Dockerfile.cpu .

finalコンテナをビルド
 $ docker build -t preprod-xgboost-container:1.5-1-cpu-py3 -f docker/1.5-1/final/Dockerfile.cpu .

構築されたイメージを確認
$ docker image ls

中に入って確認（コンテナのタグもつけて指定すること）
$ docker run -it preprod-xgboost-container:1.5-1-cpu-py3 /bin/bash

$ docker run -it 354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost:1.5-1 /bin/bash  

   

# コンテナの中に入り、 command serveを実行してみる

# モデル利用ならうまくいくのではないか？-> OK


YouTubeのリンク先ソースより

https://github.com/aws-samples/aws-ml-jp/blob/main/sagemaker/sagemaker-inference/inference-tutorial/1_sklearn.ipynb



https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model

デプロイ

https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy

ValueError: Estimator is not associated with a training job

# エラー　trainingjobとの紐付け

https://stackoverflow.com/questions/63340328/how-to-define-a-sagemaker-estimator-object-using-a-pre-trained-model-and-then-de

# END of Containts ===============

# 5.後片付け
予期せぬ課金を防ぐために、以下のリソースを削除します。

* SageMaker 推論エンドポイント
* ECR
* S3
* SageMakerノートブックインスタンス

# 参考
* SageMaker のtrainingジョブを理解する
    * https://github.com/aws-samples/aws-ml-jp/tree/main/sagemaker/sagemaker-traning/tutorial
* SageMaker-Pytorth training Toolkit
    * https://github.com/aws/sagemaker-pytorch-training-toolkit/
* SageMaker-Pytorch Inference Toolkit
    * https://github.com/aws/sagemaker-pytorch-inference-toolkit
* SageMaker Inference Toolkit
    * https://docs.aws.amazon.com/sagemaker/latest/dg/amazon-sagemaker-toolkits.html
    * https://github.com/aws/sagemaker-inference-toolkit